In [2]:
!pip3 install -q mediapipe-silicon

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks.python import vision
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import ast
import numpy as np
import os 
import pickle

objc[28132]: Class CaptureDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x106bc8860) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2854ae548). One of the two will be used. Which one is undefined.
objc[28132]: Class CVWindow is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1067d0a68) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2854ae598). One of the two will be used. Which one is undefined.
objc[28132]: Class CVView is implemented in both /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1067d0a90) and /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cv2/c

In [3]:
#open filtered data csv

data = pd.read_csv("new_data.csv")
data.head(11)

,class_name,video_id,split,bbox,link
0,fight,21704,test,"[75, 25, 218, 192]",https://www.signingsavvy.com/signs/mp4/22/2224...
1,fight,21706,train,"[81, 17, 232, 192]",https://www.signingsavvy.com/signs/mp4/22/2276...
2,find,69323,val,"[480, 69, 1350, 1080]",http://aslbricks.org/New/ASL-Videos/find.mp4
3,find,21858,train,"[79, 10, 213, 192]",https://www.signingsavvy.com/signs/mp4/6/6103.mp4
4,find,21859,train,"[68, 12, 210, 192]",https://www.signingsavvy.com/signs/mp4/9/9553.mp4
5,find,21863,train,"[187, 52, 572, 400]",http://www.aslsearch.com/signs/videos/pick.mp4
6,find,21852,train,"[30, 0, 276, 240]",https://media.spreadthesign.com/video/mp4/13/5...
7,find,21853,test,"[386, 53, 801, 720]",https://s3-us-west-1.amazonaws.com/files.start...
8,find,21854,train,"[682, 49, 1489, 1075]",https://signstock.blob.core.windows.net/signsc...
9,find,21855,test,"[112, 9, 555, 480]",https://media.asldeafined.com/vocabulary/14684...


In [7]:
 ### FUNCTION TO CREATE HANDLANDMARKS
 
def hand_land_marking(link):
  not_running = []
  mp_hands = mp.solutions.hands
  hands = mp_hands.Hands()
  mpDraw = mp.solutions.drawing_utils


  vid_path = link
  cap = cv2.VideoCapture(vid_path)

  bbox = data.loc[data['link'] == vid_path, 'bbox'].values
  bbox = ast.literal_eval(bbox[0])


  # Define the coordinates of the bounding box (top-left and bottom-right)
  x1, y1 = bbox[0], bbox[1]  # Top-left corner
  x2, y2 = bbox[2], bbox[3]  # Bottom-right corner


  if not cap.isOpened():
    print("Error: Could not open video file.")
  
    

  else:
    while cap.isOpened():
  # Read a frame from the video
        ret, frame = cap.read()
  # Check if the frame was read successfully
        if not ret:
            print("End of video.")
            break
  #Use the bbox coordinates to crop frame
        cropped_frame = frame[y1:y2, x1:x2]
        frame_rgb = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB) #recolor frame to run process

        detection = hands.process(frame_rgb)
  #If hands are detected then draw the hand landmarkd and the hand connections
        if detection.multi_hand_landmarks:
          for landmarks in detection.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame_rgb, landmarks, mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mpDraw.DrawingSpec(color=(70,130,180), thickness=-1, circle_radius=5),
                connection_drawing_spec=mpDraw.DrawingSpec(color=(0, 0, 0), thickness=1))
  #Show the image only if the hands are present
          plt.imshow(frame_rgb)
          plt.axis('off')
          plt.show()
  #Show the image even if hands not present with all the landmarks and connections
        #plt.imshow(frame_rgb)
        #plt.axis('off')
        #plt.show()


    cap.release()


In [ ]:
#test run of the hand_land_marking function
#for i in range(10):
   # hand_land_marking(data['link'][i])
import concurrent.futures

def check_video_opening(link):
    try:
        vid = cv2.VideoCapture(link)
        opened = vid.isOpened()
        vid.release()
        return opened
    except Exception as e:
        print(f"Error opening video {link}: {e}")
        return False

not_open = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(check_video_opening, link) for link in data['link']]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        if not future.result():
            not_open.append(i)

print("Videos not opened:", not_open.length)

  

[tcp @ 0x414ff21b0] Connection to tcp://www.aslsearch.com:443 failed: Operation timed out
OpenCV: Couldn't read video stream from file "http://www.aslsearch.com/signs/videos/carpet2.mp4"


In [7]:
#### WORKING CODE FOR READING IN VIDEO INTO IMAGES#########
def read_videos(link):
  count =0
  vid_path = link
  cap = cv2.VideoCapture(vid_path)
  bbox = data.loc[data['link'] == vid_path, 'bbox'].values
  bbox = ast.literal_eval(bbox[0])
  # Define the coordinates of the bounding box (top-left and bottom-right)
  x1, y1 = bbox[0], bbox[1]  # Top-left corner
  x2, y2 = bbox[2], bbox[3]  # Bottom-right corner

  # Create a hand landmarker instance with the video mode:
  #with vision.HandLandmarker.create_from_options(options) as landmarker:
  if not cap.isOpened():
    print("Error: Could not open video file.")


  else:
    while cap.isOpened():
  # Read a frame from the video

        ret, frame = cap.read()
  # Check if the frame was read successfully
        if not ret:
            print("End of video.")
            break
        frame = frame[y1:y2, x1:x2]
        rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbframe)
        plt.axis('off')
        plt.show()
        count+=1
    cap.release()
  print(count)